[筆記來源](https://www.coursera.org/learn/r-programming/lecture/pSHGL/connections-interfaces-to-the-outside-world)

R 可以<font color='red'>讀取</font>這些資料:  
- <font color='blue'> read.table, read.csv </font> 
    - 目標檔案: tablular data (有 rows 與 columns 的)。  
    - 回傳: R 的 data frame 物件。  
- <font color='blue'> read.Lines </font> 
    - 目標檔案: 文字檔。  
    - 回傳: R 的 character vector 物件。  
- <font color='blue'> source </font> 
    - 目標檔案: R 檔案。  
    - inverse of dump。  
- <font color='blue'> dget </font> 
    - 目標檔案: R 檔案。  
    - inverse of dput。  
- <font color='blue'> load </font> 
    - for reading in saved workplaces。  
    - 專門讀取 <font color='red'> binary </font> 資料。  
- <font color='blue'> unserialized </font> 
    - for reading single R objects。  
    - 專門讀取 <font color='red'> binary </font> 資料。  

當然，R 也可以<font color='red'>寫入</font>這些資料:  
- write.table  
- writeLines  
- dump  
- dput  
- save  
- serialize  

## <font color='green'> read.table </font>:
---  
最 popular 用來讀取資料的 function。

### key arguments:
- <font color='blue'> file </font>: file 的名稱(或路徑)。  
- <font color='blue'> header </font>: 邏輯子。告訴 R 這資料是否有 header(最上面的 row，也就是各個 column 代表的意思的那個 row)。  
- <font color='blue'> sep </font>: 告訴 R column 跟 column 間用甚麼區別。  
- <font color='blue'> colClasses </font>: 是一個 character vector。裡面放各個 column 的 class。  
- <font color='blue'> nrows </font>: Rows 的數量。  
- <font color='blue'> comment.char </font>: 告訴 R，哪個符號之後的句子是 comment，可以不用理會。
- <font color='blue'> skip </font>: 告訴 R 要忽略最前面的多少行。 (可能資料最前面有個介紹，可以忽略)  
- <font color='blue'> stringAsFactors </font>: 邏輯子。是否要把 <font color='red'> character variables </font> 視為 factors?

#### 讀入少量 data:
但在讀入少量資料時，可以只給 file 的名字，其他 R 都會預設好。  
R 也會自己分配適當的記憶體空間給這個資料。  
而 <font color='blue'> read.csv </font> 跟 read.table 基本上一樣，不同的點是:  
<font color='red'> read.csv 預設的 sep 是 "," [而 read.table 預設是"空白"] </font>

#### <font color='red'> 讀入大量 data </font>:
##### 此時應該在 read.table 前，先將參數設定好，老師建議:  
1. 把 [R 官方的 help page](https://www.rdocumentation.org/packages/utils/versions/3.6.2/topics/read.table) 記起來。 
2. 大約計算目標文件<font color='red'> 可能占用多少 RAM </font>。
3. 如果你的文件裡沒有註記，那就設 <font color='blue'> comment.char = ""。

##### colClasses argument 非常重要，他可以改善讀取資料的速度(幾乎是兩倍)。  
但前提是你必須<font color='red'>要知道所有 columns 的 classes </font>。  
比如所有 classes 都是 numeric，則要這樣寫: <font color='blue'> colClasses = 'numeric' </font>。  
我們可以用如下方法找出所有 classes:  

```R
initial <- read.table("datatable.txt", nrows = 100)
classes <- sapply(initial, class)
tabAll <- read.table("datatable.txt",
 colClasses = classes)
```

其中，第二行的 sapply function [在這裡有詳細的解釋](https://www.guru99.com/r-apply-sapply-tapply.html)。  
而不要忘了 <font color='blue'> class function </font> 會告訴我們這個資料的 class。  

##### <font color='red'> nrow </font> 也很重要。雖然他不會讓讀取速度變快，但 <font color='red'>可以節省記憶體空間 </font>。  
可以用 <font color='blue'> wc </font> 計算 nrow。

## <font color='green'> Textual(正文的) format </font>
---
- Dumping 跟 dputing 在建造(?) textual format 時很有用，因為創造出來的 textual format 是 <font color='red'> edit-able 且 recorvable </font>。
- 跟 read.table 或 read.csv 不同的是，dump 跟 dput 保留了 <font color='red'> metadata(後設資料) </font> [(白話文就是資料的資料)](http://techserviceslibrary.blogspot.com/2011/04/metadata.html)。
- Textual formats 在版本管理軟體如 git 發揮較好。
- Textual formats <font color='red'> 存活時間較久 </font>，因為當檔案有衝突時，較好解決。
- 遵從 "Unix Philosophy"。
- 缺點: Not space efficient。

In [2]:
y <- data.frame(a = 1, b = 'a')
dput(y) # 可以看到 output 就是 textual format(?)

structure(list(a = 1, b = "a"), class = "data.frame", row.names = c(NA, 
-1L))


In [4]:
dput(y, file="y.R") # 把 textual format 傳入 y.R 這個檔案

In [6]:
new.y <- dget("y.R") # 讀入 y.R
print(new.y)

  a b
1 1 a


## <font color='green'> Connections </font>: Interface(接口)

數據可以經由 <font color='red'> connection interfaces </font> 被讀入。
不同種的 files 有不同的 functions:
- file: (open connections to) file。
- gzfile: (open connections to) gzip 壓縮檔。
- bzfile: (open connections to) bzip2 壓縮檔。
- url: (open connections to) 網頁。
---
而上述這些通常都是在底下就做好了。比如你 read.table，R 就會 open connections 到你要的 table。  
舉例:  
```R
 con <- gzfile("words.gz")
 x <- readLines(con, 10) # 將 con 內的前 10 行存在 x 裡。
 x
 [1] "1080" "10-point" "10th" "11-point"
 [5] "12-point" "16-point" "18-point" "1st"
 [9] "2" "20-point"
```